# Simple RGB Image Plotter

Load any 3 images, choose bands, and plot as RGB composite.

In [1]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Set up matplotlib
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300

## 1. Define Image Paths

In [2]:
# Set your image paths here
image1_path = "testimages25/2025-01-08_composite_mosaic.tif"  # Red channel
image2_path = "testimages25/2025-05-25_composite_mosaic.tif"  # Green channel  
image3_path = "testimages25/2025-08-21_composite_mosaic.tif"  # Blue channel

# Or use NDWI images
# image1_path = "outputs_cleaned/2025-01-08_composite_mosaic_ndwi.tif"
# image2_path = "outputs_cleaned/2025-05-25_composite_mosaic_ndwi.tif"
# image3_path = "outputs_cleaned/2025-08-21_composite_mosaic_ndwi.tif"

print(f"Image 1 (Red): {image1_path}")
print(f"Image 2 (Green): {image2_path}")
print(f"Image 3 (Blue): {image3_path}")

Image 1 (Red): testimages25/2025-01-08_composite_mosaic.tif
Image 2 (Green): testimages25/2025-05-25_composite_mosaic.tif
Image 3 (Blue): testimages25/2025-08-21_composite_mosaic.tif


## 2. Load Images and Inspect Bands

In [3]:
def load_image(path):
    """Load image and return array + info"""
    with rasterio.open(path) as src:
        image = src.read()
        profile = src.profile
        
    print(f"\nLoaded: {path}")
    print(f"  Shape: {image.shape} (bands, height, width)")
    print(f"  Bands: {profile['count']}")
    print(f"  Data type: {profile['dtype']}")
    print(f"  Value range: {image.min():.3f} - {image.max():.3f}")
    
    return image, profile

# Load all images
img1, prof1 = load_image(image1_path)
img2, prof2 = load_image(image2_path) 
img3, prof3 = load_image(image3_path)


Loaded: testimages25/2025-01-08_composite_mosaic.tif
  Shape: (4, 5200, 4638) (bands, height, width)
  Bands: 4
  Data type: uint16
  Value range: 0.000 - 16491.000

Loaded: testimages25/2025-05-25_composite_mosaic.tif
  Shape: (4, 5200, 4638) (bands, height, width)
  Bands: 4
  Data type: uint16
  Value range: 0.000 - 14716.000

Loaded: testimages25/2025-08-21_composite_mosaic.tif
  Shape: (4, 5200, 4638) (bands, height, width)
  Bands: 4
  Data type: uint16
  Value range: 0.000 - 14757.000


## 3. Band Selection Widgets

In [11]:
# Create band selection widgets
max_bands = max(prof1['count'], prof2['count'], prof3['count'])
band_options = [(f"Band {i+1}", i) for i in range(max_bands)]

print("Select which band to use from each image:")

band1_widget = widgets.Dropdown(
    options=band_options[:prof1['count']],
    value=0,
    description='Red (Img1):'
)

band2_widget = widgets.Dropdown(
    options=band_options[:prof2['count']], 
    value=0,
    description='Green (Img2):'
)

band3_widget = widgets.Dropdown(
    options=band_options[:prof3['count']],
    value=0, 
    description='Blue (Img3):'
)

display(widgets.VBox([band1_widget, band2_widget, band3_widget]))

Select which band to use from each image:


## 4. Normalization Options

In [12]:
# Normalization widgets
print("Normalization options:")

norm_method = widgets.Dropdown(
    options=[
        ('Min-Max (0-1)', 'minmax'),
        ('Percentile (2-98%)', 'percentile'),
        ('Standard deviation (±2σ)', 'std'),
        ('No normalization', 'none')
    ],
    value='percentile',
    description='Method:'
)

display(norm_method)

Normalization options:


Dropdown(description='Method:', index=1, options=(('Min-Max (0-1)', 'minmax'), ('Percentile (2-98%)', 'percent…

## 5. RGB Plotting Function

In [13]:
def normalize_band(band, method='percentile'):
    """Normalize a band for display"""
    band = band.astype(np.float32)
    
    if method == 'minmax':
        return (band - band.min()) / (band.max() - band.min())
    
    elif method == 'percentile':
        p2, p98 = np.percentile(band, [2, 98])
        return np.clip((band - p2) / (p98 - p2), 0, 1)
    
    elif method == 'std':
        mean, std = band.mean(), band.std()
        return np.clip((band - mean + 2*std) / (4*std), 0, 1)
    
    else:  # no normalization
        return band

def plot_rgb_composite():
    """Create and plot RGB composite"""
    # Get selected bands
    red_band = img1[band1_widget.value]
    green_band = img2[band2_widget.value] 
    blue_band = img3[band3_widget.value]
    
    # Normalize bands
    method = norm_method.value
    red_norm = normalize_band(red_band, method)
    green_norm = normalize_band(green_band, method)
    blue_norm = normalize_band(blue_band, method)
    
    # Stack into RGB
    rgb = np.stack([red_norm, green_norm, blue_norm], axis=2)
    
    # Plot
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    
    # Individual bands
    axes[0].imshow(red_norm, cmap='Reds')
    axes[0].set_title(f'Red: {image1_path.split("/")[-1]}\nBand {band1_widget.value + 1}')
    axes[0].axis('off')
    
    axes[1].imshow(green_norm, cmap='Greens')
    axes[1].set_title(f'Green: {image2_path.split("/")[-1]}\nBand {band2_widget.value + 1}')
    axes[1].axis('off')
    
    axes[2].imshow(blue_norm, cmap='Blues')
    axes[2].set_title(f'Blue: {image3_path.split("/")[-1]}\nBand {band3_widget.value + 1}')
    axes[2].axis('off')
    
    # RGB composite
    axes[3].imshow(rgb)
    axes[3].set_title(f'RGB Composite\nNormalization: {method}')
    axes[3].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Composite Info:")
    print(f"   Red channel: {image1_path.split('/')[-1]} (Band {band1_widget.value + 1})")
    print(f"   Green channel: {image2_path.split('/')[-1]} (Band {band2_widget.value + 1})")
    print(f"   Blue channel: {image3_path.split('/')[-1]} (Band {band3_widget.value + 1})")
    print(f"   Normalization: {method}")
    print(f"   Output shape: {rgb.shape}")

## 6. Interactive Plot Button

In [14]:
# Create plot button
plot_button = widgets.Button(
    description='Create RGB Plot',
    button_style='success',
    icon='check'
)

def on_plot_click(b):
    plot_rgb_composite()

plot_button.on_click(on_plot_click)

display(plot_button)

Button(button_style='success', description='Create RGB Plot', icon='check', style=ButtonStyle())

## 7. Quick Presets

In [9]:
def set_preset(preset_name):
    """Set common band combinations"""
    if preset_name == 'true_color':
        # Assuming Planet bands: R=0, G=1, B=2, NIR=3
        band1_widget.value = 0  # Red
        band2_widget.value = 1  # Green  
        band3_widget.value = 2  # Blue
        print("Set to True Color: R-G-B")
        
    elif preset_name == 'false_color':
        band1_widget.value = 3  # NIR -> Red
        band2_widget.value = 0  # Red -> Green
        band3_widget.value = 1  # Green -> Blue  
        print("Set to False Color: NIR-R-G")
        
    elif preset_name == 'ndwi_only':
        # If using NDWI images (single band)
        band1_widget.value = 0
        band2_widget.value = 0
        band3_widget.value = 0
        print("Set to NDWI grayscale")

# Preset buttons
preset_buttons = [
    widgets.Button(description='True Color (R-G-B)', button_style='info'),
    widgets.Button(description='False Color (NIR-R-G)', button_style='info'),
    widgets.Button(description='NDWI Grayscale', button_style='info')
]

preset_buttons[0].on_click(lambda b: set_preset('true_color'))
preset_buttons[1].on_click(lambda b: set_preset('false_color'))
preset_buttons[2].on_click(lambda b: set_preset('ndwi_only'))

print("Quick presets:")
display(widgets.HBox(preset_buttons))

Quick presets:


## 8. Save RGB Composite

In [10]:
def save_rgb_composite(filename="rgb_composite.png", dpi=300):
    """Save the current RGB composite"""
    # Get selected bands and create composite
    red_band = img1[band1_widget.value]
    green_band = img2[band2_widget.value] 
    blue_band = img3[band3_widget.value]
    
    method = norm_method.value
    red_norm = normalize_band(red_band, method)
    green_norm = normalize_band(green_band, method)
    blue_norm = normalize_band(blue_band, method)
    
    rgb = np.stack([red_norm, green_norm, blue_norm], axis=2)
    
    # Save
    plt.figure(figsize=(12, 8))
    plt.imshow(rgb)
    plt.axis('off')
    plt.title(f'RGB Composite: R={band1_widget.value+1}, G={band2_widget.value+1}, B={band3_widget.value+1}')
    plt.savefig(filename, dpi=dpi, bbox_inches='tight', pad_inches=0)
    plt.close()
    
    print(f"✅ Saved RGB composite to: {filename}")

# Save button
save_button = widgets.Button(description='Save RGB', button_style='warning')
save_button.on_click(lambda b: save_rgb_composite())

display(save_button)

Button(button_style='warning', description='Save RGB', style=ButtonStyle())